### Importing the libraries

In [ ]:
import os
import yaml
import pickle
import networkx as nx
import matplotlib.pyplot as plt

import MCFS

from MCFS.utils import *
from MCFS.isolines import *
from MCFS.MMRTC import *
from MCFS.isograph import *
from MCFS.planning import Pathfinder, unified_CFS
from MCFS.solution_refinement import *

### 1.  Original CFS v.s. our unified CFS 

In [ ]:
instance_name = "double_torus"

with open(os.path.join("data", "instances", instance_name+".yaml")) as f:
    dict = yaml.load(f, yaml.Loader)
    with open(f"data/polygons/{dict['polygon']}", "rb") as f:
        polygon = pickle.load(f)

layered_isolines = LayeredIsoLines.from_polygon(polygon, 0.15)
IG = IsoGraph.build(layered_isolines)

dat = {"bidir": unified_CFS(
                    G = IG,
                    r = layered_isolines.at(0, 0),
                    pr_idx = 0,
                    selector_type = "MCS"
                )[0],
}

IG = IsoGraph.build(layered_isolines, bidirection=False)
dat["unidir"] = \
    unified_CFS(
        G = IG,
        r = layered_isolines.at(0, 0),
        pr_idx = 0,
        selector_type = "MCS"
    )[0]


fig, ax = plt.subplots(1, 2, figsize=(8, 4), dpi=200)

def subplot(pi, ax):
    pi = np.array(pi)
    ax.plot(pi[:, 0], pi[:, 1], f'-k')
    ax.axis("off")
    ax.set_xlim([-2, 2])
    ax.set_ylim([-4, 4])
    ax.axis("equal")

subplot(np.array(dat["unidir"]), ax[0])
subplot(np.array(dat["bidir"]), ax[1])

plt.subplots_adjust(wspace=0)


### 2. Demonstrating the 2 MMRTC solution optimizations

#### Defining the drawing functions

In [ ]:
colors = ["r", "b", "k", "c", "g", "m"]
shapes = ["s", "^", "X", "+"]

def plot_MMRTC_sol_nonref(IG, R, T, dups):
    fig, ax = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(10, 3), dpi=200)
    W = sorted([IG.nodes[v]["weight"] for v in IG.nodes])
    norm_ms = lambda w: 8 * (1 + (w - W[0])/(W[-1] - W[0]))
    for i in range(len(T)):
        for u, v in T[i].edges:
            pos_u = T[i].nodes[u]["pos"]
            pos_v = T[i].nodes[v]["pos"]
            ax[i+1].plot([pos_u[0], pos_v[0]], [pos_u[1], pos_v[1]], f"-{colors[i%len(colors)]}", lw=2, mew=3)
        
        for v in T[i].nodes:
            pos = T[i].nodes[v]["pos"]
            weight = T[i].nodes[v]["weight"]
            if v.index in dups[i]:
                ax[i+1].plot(pos[0], pos[1], f"o{colors[i%len(colors)]}", ms=norm_ms(weight), lw=2, mew=3)
            else:
                ax[i+1].plot(pos[0], pos[1], f"o{colors[i%len(colors)]}", ms=norm_ms(weight), mfc='none', lw=2, mew=3)

        ax[i+1].axis("on")
        ax[i+1].set_xlim([-0.25, 1.25])
        ax[i+1].set_ylim([-0.7, 6.3])
        ax[i+1].annotate(f"$r_{i+1}$", xy=(0.07,-0.1), xycoords='data',
                   xytext=(.45, .11), textcoords='axes fraction', va='top', ha='right', fontsize=18,
                   arrowprops={'arrowstyle':"->", 'connectionstyle':"arc3"})
        
    for u, v in IG.edges:
        pos_u = IG.nodes[u]["pos"]
        pos_v = IG.nodes[v]["pos"]
        ax[0].plot([pos_u[0], pos_v[0]], [pos_u[1], pos_v[1]], f"-k", lw=2, mew=3)
    for v in IG.nodes:
        pos = IG.nodes[v]["pos"]
        weight = IG.nodes[v]["weight"]
        ax[0].plot(pos[0], pos[1], 'ok', mfc='none', ms=norm_ms(weight), lw=2, mew=3)

    ax[0].axis("on")
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].set_xlim([-0.25, 1.25])
    ax[0].set_ylim([-0.7, 6.3])
    # ax[0].annotate(f"$r_1, r_2, r_3, r_4$", xy=(0.07,-0.1), xycoords='data',
    #                xytext=(.75, .1), textcoords='axes fraction', va='top', ha='right', 
    #                arrowprops={'arrowstyle':"->", 'connectionstyle':"arc3"})
    fig.tight_layout()


def plot_MMRTC_sol_ref(IG, R, T):
    fig, ax = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(10, 3), dpi=200)
    W = sorted([IG.nodes[v]["weight"] for v in IG.nodes])
    norm_ms = lambda w: 8 * (1 + (w - W[0])/(W[-1] - W[0]))
    for i in range(len(T)):
        for u, v in T[i].edges:
            pos_u = T[i].nodes[u]["pos"]
            pos_v = T[i].nodes[v]["pos"]
            ax[i+1].plot([pos_u[0], pos_v[0]], [pos_u[1], pos_v[1]], f"-{colors[i%len(colors)]}", lw=2, mew=3)
        
        j = 0
        shape_dict = {}
        for v in T[i].nodes:
            pos = T[i].nodes[v]["pos"]
            weight = T[i].nodes[v]["weight"]
            if T[i].nodes[v].get("origin"):
                ax[i+1].plot(pos[0], pos[1], f"{shapes[j]}{colors[i%len(colors)]}", ms=norm_ms(weight), mfc='none', lw=2, mew=3)
                shape_dict[T[i].nodes[v]["origin"][0].index] = shapes[j]
                shape_dict[T[i].nodes[v]["origin"][1].index] = shapes[j]
                j += 1
            else:
                ax[i+1].plot(pos[0], pos[1], f"o{colors[i%len(colors)]}", ms=norm_ms(weight), mfc='none', lw=2, mew=3)

        ax[i+1].axis("on")
        ax[i+1].set_xlim([-0.25, 1.25])
        ax[i+1].set_ylim([-0.7, 6.3])
        ax[i+1].annotate(f"$r_{i+1}$", xy=(0.25,0.5), xycoords='data',
                   xytext=(.375, .1), textcoords='axes fraction', va='top', ha='right', fontsize=18,
                   arrowprops={'arrowstyle':"->", 'connectionstyle':"arc3"})
        
    for u, v in IG.edges:
        pos_u = IG.nodes[u]["pos"]
        pos_v = IG.nodes[v]["pos"]
        ax[0].plot([pos_u[0], pos_v[0]], [pos_u[1], pos_v[1]], f"-k", lw=2, mew=3)
    for v in IG.nodes:
        pos = IG.nodes[v]["pos"]
        weight = IG.nodes[v]["weight"]
        if v.index in shape_dict:
            ax[0].plot(pos[0], pos[1], f'{shape_dict[v.index]}k', mfc='none', ms=norm_ms(weight), lw=2, mew=3)
        else:
            ax[0].plot(pos[0], pos[1], 'ok', mfc='none', ms=norm_ms(weight), lw=2, mew=3)

    ax[0].axis("on")
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].set_xlim([-0.25, 1.25])
    ax[0].set_ylim([-0.7, 6.3])
    # ax[0].annotate(f"$r_1, r_2, r_3, r_4$", xy=(0.07,-0.1), xycoords='data',
    #                xytext=(.75, .1), textcoords='axes fraction', va='top', ha='right', 
    #                arrowprops={'arrowstyle':"->", 'connectionstyle':"arc3"})
    fig.tight_layout()



#### Running all MCFS variants and drawing the results

In [ ]:
instance_name = "P"

with open(os.path.join("data", "instances", instance_name+".yaml")) as f:
    dict = yaml.load(f, yaml.Loader)
    delta = int(dict["delta"])
    interval = float(dict["interval"])
    R = [tuple(r) for r in dict["R"]]
    with open(f"data/polygons/{dict['polygon']}", "rb") as f:
        polygon = pickle.load(f)

def MCFS_NONE() -> Tuple[nx.Graph, list, List[nx.Graph]]:
    layered_isolines = LayeredIsoLines.from_polygon(polygon, interval)
    _R = [layered_isolines.at(*index) for index in R]
    IG = IsoGraph.build(layered_isolines)
    return IG, _R, solve_MMRTC_model_GRB(IG, _R)

def MCFS_AUG() -> Tuple[nx.Graph, list, List[nx.Graph]]:
    layered_isolines = LayeredIsoLines.from_polygon(polygon, interval)
    _R = [layered_isolines.at(*index) for index in R]
    IG = IsoGraph.build(layered_isolines)
    IG = augmented_isograph(IG, interval=interval, delta=delta)
    return IG, _R, solve_MMRTC_model_GRB(IG, _R)

def MCFS_REF() -> Tuple[nx.Graph, list, List[nx.Graph]]:
    layered_isolines = LayeredIsoLines.from_polygon(polygon, interval)
    _R = [layered_isolines.at(*index) for index in R]
    IG = IsoGraph.build(layered_isolines)
    T = solve_MMRTC_model_GRB(IG, _R)
    return IG, _R, solution_refinement(IG, T, _R, Pathfinder(IG, interval))

def MCFS_BOTH() -> Tuple[nx.Graph, list, List[nx.Graph]]:
    layered_isolines = LayeredIsoLines.from_polygon(polygon, interval)
    _R = [layered_isolines.at(*index) for index in R]
    IG = IsoGraph.build(layered_isolines)
    IG = augmented_isograph(IG, interval=interval, delta=delta)
    T = solve_MMRTC_model_GRB(IG, _R)
    return IG, _R, solution_refinement(IG, T, _R, Pathfinder(IG, interval))

dat = {
    'NONE': MCFS_NONE(),
    'AUG': MCFS_AUG(),
    'REF': MCFS_REF(),
    'BOTH': MCFS_BOTH()
}

plot_MMRTC_sol_nonref(*dat["NONE"],
    dups = [
            [(0, 0), (1, 0), (2, 0), (3, 0)],
            [(0, 0)],
            [(0, 0), (1, 0), (2, 0)],
            [(0, 0), (1, 0), (2, 0), (3, 0)]]
)
plot_MMRTC_sol_nonref(*dat["AUG"],
    dups = [
            [(0, 0)],
            [(0, 0)],
            [(0, 0)],
            [(0, 0)]]
    )
plot_MMRTC_sol_ref(*dat["BOTH"])
